<a href="https://colab.research.google.com/github/Yuvarajuu/Fraud-detection/blob/main/fakenewsdetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
df_fake=pd.read_csv('/content/Fake.csv')
df_real=pd.read_csv('/content/True.csv')
df_fake['label']="Fake"
df_real['label']="Real"

In [15]:
data=pd.concat([df_fake,df_real])
data=data.sample(frac=1).reset_index(drop=True)
data

,title,text,subject,date,label
0,Senate blocks Republican plan on 'watch list' ...,WASHINGTON (Reuters) - The U.S. Senate on Mond...,politicsNews,"June 20, 2016",Real
1,U.S. State Department criticized over quiet re...,WASHINGTON (Reuters) - The U.S. State Departme...,politicsNews,"March 3, 2017",Real
2,WOW! Hillary’s VP Pick Tim Kaine Gets Only 30 ...,BREAKING: #TimKaine rally only about 30 people...,left-news,"Oct 25, 2016",Fake
3,"BREAKING! Ammon, Ryan Bundy, 5 Fellow Patriots...",Who could forget the unbelievable standoff in ...,Government News,"Oct 27, 2016",Fake
4,Austria's prospective ruling parties agree on ...,VIENNA (Reuters) - Austria s conservative Peop...,worldnews,"November 3, 2017",Real
...,...,...,...,...,...
44893,British PM May vows to stay as party plotters ...,LONDON (Reuters) - British Prime Minister Ther...,worldnews,"October 6, 2017",Real
44894,Trump aide dismisses audio tape as Democrats r...,"WASHINGTON, 2016 - A top aide to Donald Trump ...",politicsNews,"May 15, 2016",Real
44895,CATHOLIC PRIEST GOES OFF On Partial-Birth Abor...,Father Gerald Murphy goes off on Hillary Clint...,politics,"Oct 21, 2016",Fake
44896,Trudeau sees no flood of Americans to Canada i...,WASHINGTON (Reuters) - Canadian Prime Minister...,politicsNews,"March 11, 2016",Real


In [17]:
x=data['text']
y=data['label']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [20]:
tfidf=TfidfVectorizer(stop_words="english")
x_train=tfidf.fit_transform(x_train)
x_test=tfidf.transform(x_test)

In [21]:
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [22]:
pre=model.predict(x_test)
classification_report(y_test, pre)

'              precision    recall  f1-score   support\n\n        Fake       0.99      0.99      0.99      4741\n        Real       0.98      0.99      0.99      4239\n\n    accuracy                           0.99      8980\n   macro avg       0.99      0.99      0.99      8980\nweighted avg       0.99      0.99      0.99      8980\n'

In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses
import numpy as np

In [54]:
x=data['text'].astype(str).values
y=data['label'].values
x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,test_size=0.2)
y_train1 = np.where(y_train1 == 'Real', 1, 0)
y_test1 = np.where(y_test1 == 'Real', 1, 0)

In [55]:
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(x_train1)
x_train_seq=tokenizer.texts_to_sequences(x_train1)
x_test_seq=tokenizer.texts_to_sequences(x_test1)
x_train_pad=pad_sequences(x_train_seq,maxlen=200)
x_test_pad=pad_sequences(x_test_seq,maxlen=200)

In [45]:
from tensorflow.keras import regularizers


In [56]:
model=Sequential()
model.add(Embedding(input_dim=20000,output_dim=128,input_length=200))
model.add(GRU(128,dropout=0.5,recurrent_dropout=0.5))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid',kernel_regularizer=regularizers.l2(0.01)))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [57]:
model.compile(optimizer='adam',loss=losses.BinaryCrossentropy(),metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [58]:
model.fit(x_train_pad,y_train1,epochs=5,batch_size=128,validation_split=0.2)

Epoch 1/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 293s 1s/step - accuracy: 0.8393 - loss: 0.3883 - val_accuracy: 0.9655 - val_loss: 0.1099
Epoch 2/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 314s 1s/step - accuracy: 0.9771 - loss: 0.0811 - val_accuracy: 0.9723 - val_loss: 0.0952
Epoch 3/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 296s 1s/step - accuracy: 0.9825 - loss: 0.0613 - val_accuracy: 0.9823 - val_loss: 0.0743
Epoch 4/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 317s 1s/step - accuracy: 0.9907 - loss: 0.0378 - val_accuracy: 0.9811 - val_loss: 0.0662
Epoch 5/5
225/225 ━━━━━━━━━━━━━━━━━━━━ 323s 1s/step - accuracy: 0.9926 - loss: 0.0278 - val_accuracy: 0.9851 - val_loss: 0.0631


In [59]:
loss, acc = model.evaluate(x_test_pad, y_test1, verbose=1)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {acc:.4f}')

281/281 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - accuracy: 0.9847 - loss: 0.0544
Test Loss: 0.0600, Test Accuracy: 0.9834


In [60]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")
model.save("fake_news_model.h5")


In [65]:
from google.colab import files
files.download("fake_news_model.h5")
files.download("tokenizer.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>